# Calculate distance from cities in UCDB dataset

1. Distance to shoreline (shoreline is the OCEAN MASK dataset from the official World Bank boundaries)
2. Distance to border (from WB GAD)

In [ ]:
import sys
import os

import geopandas as gpd
import pandas as pd


sys.path.insert(0, "C:/WBG/Work/Code/GOSTrocks/src")

import GOSTrocks.misc as misc
from GOSTrocks.misc import tPrint

%load_ext autoreload
%autoreload 2

In [ ]:
data_folder = r"C:\WBG\Work\data\URBAN"

ucdb_file = os.path.join(data_folder, "GHS_UCDB_GLOBE_R2024A.gpkg")
# fua_file = os.path.join(data_folder, "GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg")
# fua_peripheries = os.path.join(data_folder, "FUA_peripheries.gpkg")
# oxford_extents = os.path.join(data_folder, "OE_FUA_SHAPEFILE", "OE_FUA_SHAPEFILE.shp")

ucdb_id = "ID_UC_G0"

coastline_file = r"C:\WBG\Work\data\ADMIN\BETTER_FORMATS\ocean_mask.gpkg"
border_file = r"C:\WBG\Work\data\ADMIN\BETTER_FORMATS\WB_GAD_Lines.gpkg"

out_folder = r"C:\WBG\Work\Projects\MENA_Urban\RESULTS\DISTANCE"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

In [ ]:
inU = gpd.read_file(ucdb_file)
inU = inU.loc[:, [ucdb_id, "geometry"]]
inU.head()

In [ ]:
inCoasts = gpd.read_file(coastline_file)
# inCoasts['geometry'] = inCoasts.geometry.buffer(0)
inCoasts = misc.explodeGDF(inCoasts)
inCoasts["geometry"] = inCoasts["geometry"].boundary
inCoasts.drop(["level_0", "index", "id"], inplace=True, axis=1, errors="ignore")
inCoasts = misc.explodeGDF(inCoasts)
inCoasts.drop(["level_0", "index", "id"], inplace=True, axis=1, errors="ignore")
inCoasts["length"] = inCoasts.geometry.length
inCoasts = inCoasts.iloc[1:]
inCoasts = inCoasts.to_crs(inU.crs)
inCoasts.to_file(os.path.join(out_folder, "coastlines.gpkg"), driver="GPKG")

In [ ]:
inBorders = gpd.read_file(border_file)
inBorders.explode(index_parts=True, ignore_index=True, inplace=True)
inBorders["ID"] = list(range(0, len(inBorders)))
inBorders = inBorders.to_crs(inU.crs)
inBorders = misc.explodeGDF(inBorders)
inBorders.drop(["level_0", "index"], inplace=True, axis=1, errors="ignore")

In [ ]:
tPrint("Started")
inU_borders = gpd.sjoin_nearest(inU, inBorders, how="left", distance_col="BORDER_DIST")
tPrint("Finished border distances")
inU_coasts = gpd.sjoin_nearest(inU, inCoasts, how="left", distance_col="COAST_DIST")
tPrint("Finished coastline distances")

In [ ]:
inU_combined = pd.merge(inU_borders, inU_coasts[[ucdb_id, "COAST_DIST"]], on=ucdb_id)
inU_combined.to_file(
    os.path.join(out_folder, "UCDB_distances.gpkg"), driver="GPKG", index=False
)
inU_combined[["ID_UC_G0", "BORDER_DIST", "COAST_DIST"]].to_csv(
    os.path.join(out_folder, "UCDB_distances.csv"), index=False
)

In [ ]:
inU.shape